In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision.transforms.functional import pil_to_tensor
import torch.nn.functional as F
from torch.optim.adam import Adam

In [2]:
train_data = MNIST(root="./data", download=True, train=True)
val_data = MNIST(root="./data", download=True, train=False)

In [3]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1):
        super().__init__()
        self.conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = EncoderBlock(1, 16)
        self.block2 = EncoderBlock(16, 32)
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        return x
    
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = DecoderBlock(32, 16)
        self.block2 = DecoderBlock(16, 8)
        self.conv_output = nn.Conv2d(8, 1, kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.conv_output(x)
        return x
    

class VQEmbedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, beta: float = 0.25):
        super().__init__()

        self.beta = beta

        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/num_embeddings, 1/num_embeddings)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, c, h, w = x.shape
        z = x.permute(0, 2, 3, 1)
        z = torch.reshape(z, (b * h * w, c))

        distance = (
            torch.sum(z**2, dim=1, keepdim=True) + \
            torch.sum(self.embedding.weight ** 2, dim=0, keepdim=True) + \
            -2 * torch.matmul(z, self.embedding.weight)
        )

        idx = torch.argmin(distance, dim=1)
        e = self.embedding(idx)

        st_loss = F.mse_loss(e, z.detach()) + self.beta * F.mse_loss(e.detach(), z)

        # straight-through estimator
        z_q = z + (e - z).detach()

        z_q = torch.reshape(z_q, (b, h, w, c))
        z_q = torch.permute(z_q, (0, 3, 1, 2))

        return z_q, st_loss


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = Decoder()
        self.encoder = Encoder()
        self.embeddings = VQEmbedding(num_embeddings=32, embedding_dim=32)
    
    def forward(self, x):
        h = self.encoder(x)
        z_q, st_loss = self.embeddings(h)
        output = self.decoder(z_q)
        output = F.sigmoid(output)

        return output, st_loss

In [5]:
from torch.utils.data import DataLoader, Dataset

class MNISTDataset(Dataset):
    def __init__(self, train):
        self.data = MNIST(root="./data", download=True, train=train)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img = self.data[index][0]

        img_tensor = pil_to_tensor(img)
        img_tensor = img_tensor / 255.0

        return img_tensor

In [6]:
device = "cuda"
batch_size = 1024
lr = 1e-2
seed = 42

In [7]:
torch.manual_seed(seed)
model = Model()
model = model.to(device)

In [8]:
train_data = MNISTDataset(train=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

val_data = MNISTDataset(train=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

In [9]:
optimizer = Adam(model.parameters(), lr=lr)

In [10]:
num_epochs = 10

for epoch in range(num_epochs):
    train_epoch_loss, val_epoch_loss = 0.0, 0.0
    for imgs in train_loader:
        model.zero_grad()

        imgs = imgs.to(device)
        o, st_loss = model(imgs)

        reconstruction_loss = F.mse_loss(o, imgs)
        loss = reconstruction_loss + st_loss
        loss.backward()
        optimizer.step()

        train_epoch_loss += loss.detach().cpu().numpy()

    train_epoch_loss = round(train_epoch_loss / len(train_loader), 4)

    with torch.no_grad():
        for img in val_loader:
            imgs = imgs.to(device)
            o, st_loss = model(imgs)

            reconstruction_loss = F.mse_loss(o, imgs)
            loss = reconstruction_loss + st_loss

            val_epoch_loss += loss.detach().cpu().numpy()

    val_epoch_loss = round(val_epoch_loss / len(val_loader), 4)


    print(f"----- Epoch {epoch} -----")
    print(f"Training loss: {train_epoch_loss}")
    print(f"Val loss: {val_epoch_loss}")

----- Epoch 0 -----
Training loss: 0.4620000123977661
Val loss: 0.2752000093460083
----- Epoch 1 -----
Training loss: 0.18320000171661377
Val loss: 0.09960000216960907
----- Epoch 2 -----
Training loss: 0.08020000159740448
Val loss: 0.06419999897480011
----- Epoch 3 -----
Training loss: 0.057500001043081284
Val loss: 0.052299998700618744
----- Epoch 4 -----
Training loss: 0.049400001764297485
Val loss: 0.047200001776218414
----- Epoch 5 -----
Training loss: 0.047600001096725464
Val loss: 0.049800001084804535
----- Epoch 6 -----
Training loss: 0.049400001764297485
Val loss: 0.04749999940395355
----- Epoch 7 -----
Training loss: 0.04520000144839287
Val loss: 0.04340000078082085
----- Epoch 8 -----
Training loss: 0.04179999977350235
Val loss: 0.04129999876022339
----- Epoch 9 -----
Training loss: 0.0421999990940094
Val loss: 0.04439999908208847
